In [1]:
import sys
sys.path.append('../')

from spd.MajoranaRepresentation import MajoranaRepresentation
from spd.SparsePauliDynamics import Simulation
from spd.utils import *

import numpy as np
import matplotlib.pyplot as plt
from functools import partial

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import NumPyMinimumEigensolver

In [2]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
problem = driver.run()
hamiltonian = problem.hamiltonian.second_q_op()
obs = hamiltonian
hamiltonian0 = FermionicOp.from_terms([term for term in problem.hamiltonian.second_q_op().terms() if len(term[0])==2])
hamiltonian1 = (hamiltonian - hamiltonian0).simplify()

dt = 0.01
nsteps = 1000
threshold = 0

In [3]:
observable = MajoranaRepresentation.from_fermionic_op(obs)
h0 = MajoranaRepresentation.fermionic_to_sparse_pauli_op(hamiltonian0)
h1 = MajoranaRepresentation.fermionic_to_sparse_pauli_op(hamiltonian1)
hf_state = np.array(HartreeFock(problem.num_spatial_orbitals, problem.num_particles, JordanWignerMapper())._bitstr)

def ham_func(step):
    return dt * (h0 + (1-(step+1)/nsteps) * h1)
def eval_exp_val(obs):
    return partial(MajoranaRepresentation.exp_val_comp_basis_state, state=hf_state)(obs)

sim = Simulation(observable, dt*h0, threshold=threshold)
r_majorana = sim.run_dynamics(nsteps, process=eval_exp_val, process_every=1, td_ham=ham_func)

In [4]:
energy_asp = r_majorana[-1] + problem.hamiltonian.nuclear_repulsion_energy

## Exact simulation

In [5]:
mapper = JordanWignerMapper()
algo = NumPyMinimumEigensolver()
algo.filter_criterion = problem.get_default_filter_criterion()
solver = GroundStateEigensolver(mapper, algo)
result = solver.solve(problem)
energy_exact = result.total_energies[0]

## Compare

In [6]:
energy_asp - energy_exact

(0.0002725567817796204+0j)